In [7]:
from IPython.display import display, HTML,clear_output
import random as rnd
import time

In [8]:
class SVGCanvas:
    def __init__(self,width,height,size):
        self.width = width
        self.height = height
        self.size = size
        self.html_s =""
    
    def clear(self):
        self.html_s = '<svg width="'+str(self.width)+'" height="'+str(self.height)+'"style="border:1px solid black">'

    def circle(self,x_pos,y_pos,color):
        x_pos = x_pos/self.size
        y_pos = y_pos/self.size
        r = 1/(self.size*2)
        str_int_x = str(int((x_pos*self.width)+(r*self.width)))
        str_int_y = str(int((y_pos*self.height)+(r*self.width)))
        str_int_r = str(int(r*self.width))
        self.html_s +='<circle cx="'+str_int_x+'" cy="'+str_int_y+'" r="'+str_int_r+'" fill="'+color+'"/>'
    
    def rect(self,x_pos,y_pos,color):
        x_pos = x_pos/self.size
        y_pos = y_pos/self.size
        height = 1/self.size
        width = 1/self.size
        str_int_x = str(int(x_pos*self.width))
        str_int_y = str(int(y_pos*self.height))
        str_int_width = str(int(width*self.width))
        str_int_height = str(int(height*self.height))
        self.html_s +='<rect x="'+str_int_x+'" y="'+str_int_y+'" width="'+str_int_width+'" height="'+str_int_height+'" fill="'+color+'"/>'

    def getCanvas(self):
        self.html_s += "</svg>"
        return HTML(self.html_s)

In [9]:
class Counter:
    def __init__(self):
        self.tnow =0
        self.tnext = 1
        
    def update(self):
        # This switches the 0 1 state of now and next
        self.tnow = (self.tnow+1) % 2
        self.tnext = (self.tnext+1) % 2

In [10]:
class Cell:
    max_resource = 10
    def __init__(self,x_pos,y_pos,svg,counter):
        self.neighbours = []
        self.number_of_neighbours=-1
        self.x_pos = x_pos
        self.y_pos = y_pos
        self.svg = svg
        self.resources = [0.0,0.0]
        self.counter = counter
        self.occupant = None
        
    def addNeighbour(self,cell):
        self.neighbours.append(cell)
        self.number_of_neighbours+=1
    
    def randomNeighbour(self):
        return rnd.choice(self.neighbours)
    
    @property
    def resource(self):
        return self.resources[self.counter.tnow]
    
    @property
    def resource_rgb(self):
        return str(int(255-(self.resource*255)/Cell.max_resource))
    
    @property
    def empty(self):
        return self.occupant == None
    
    def draw(self):
        self.svg.rect(self.x_pos,self.y_pos,'rgb(255,'+self.resource_rgb+','+self.resource_rgb+')')
        if self.occupant !=None:
            self.svg.circle(self.x_pos,self.y_pos,'blue')

                

In [11]:
class Agent:
    def __init__(self,home,counter):
        self.home = home
        self.home.occupant = self
        self.counter = counter
    
    def randomMove(self):
        move_to = self.home.randomNeighbour()
        if move_to.empty:
            self.home.occupant = None
            self.home = move_to
            self.home.occupant = self
    

In [12]:
class Experiment:
    def __init__(self,size):        
        self.cells = []
        self.agents =[]
        self.size = size
        self.total_cells = size*size
        self.svg = SVGCanvas(300,300,size)
        

    def setUp(self,agents):
        self.cells = []
        self.agents = []
        self.counter = Counter()
        
        for i in range(self.size*self.size):
            self.cells.append(Cell(int(i/self.size),(i%self.size),self.svg,self.counter))
        for cell in self.cells:
            for  y in range(cell.y_pos-1,cell.y_pos+2):
                for  x in range(cell.x_pos-1,cell.x_pos+2):
                    x=self.bounds(x)
                    y=self.bounds(y)
                    pos = x*self.size+y
                    cell.addNeighbour(self.cells[pos])
        for _ in range(agents):
            home = rnd.choice(self.cells)
            if home.empty:
                agent = Agent(home,self.counter)
                self.agents.append(agent)
        
    def bounds(self,i):
        if i<0:
            return self.size + i 
        if i>=self.size:
            return i-self.size
        return i
        
    def iterate(self):
        rnd.shuffle(self.agents)
        for agent in self.agents:
            agent.randomMove()
        self.draw()
        
    def draw(self):
        self.svg.clear()
        for cell in self.cells:
            cell.draw()
        clear_output(wait=True)
        display(self.svg.getCanvas())


In [13]:
experiment = Experiment(32)
experiment.setUp(40)
for _ in range(100):
    experiment.iterate()
    time.sleep(0.1)
print("this done")


done
